<a href="https://colab.research.google.com/github/nisanuro/CNG483-Project1/blob/master/CNG483_Project1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [346]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from google.colab import output
from google.colab import drive
import concurrent.futures
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def read_images(path):
    images = []

    for filename in os.listdir(path):
        img=cv2.imread(os.path.join(image_path, filename),cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        images.append(img)

    return images

In [0]:
"""def histogram(image, b):
    bins = []
    width = int(256/b)

    for low in range(0, b * width, width):
        bins.append((low, low+width))

    hist = np.zeros(b).astype('int32')
    image = np.array(image).flatten()

    for value in image:
        for i in range(0, len(bins)):
            if (bins[i][0] <= value < bins[i][1]):
                hist[i] += 1
    
    return hist"""

def histogram(image,b):
    bins = np.array([i for i in range(0, 256, int(256/b))])
    image = np.array(image).flatten()
    inds = np.digitize(image,bins, right=False)
    count = np.bincount(inds)
    #print("count: ")
    #print(len(count))
    #count = count[::int(256/b)]
    #print(count)
    #print(bins)
    return count[1:]

In [0]:
def color_histogram(img, b):
    chans = cv2.split(img)
    colors = ("b", "g", "r")

    features = []
   
    for (chan, color) in zip(chans, colors):
        if color == 'b':
            bf = histogram(chan, b)
        elif color == 'g':
            gf = histogram(chan, b)
        else:
            rf = histogram(chan, b)
    
    # combination of blue and red channel bins
    temp = np.transpose([np.tile(bf, len(gf)), np.repeat(gf, len(bf))])
    
    blue_green_hist = []    
    for i in temp:
        blue_green_hist.append(sum(i))

    # same operation with red channel bins
    temp = np.transpose([np.tile(rf, len(blue_green_hist)), np.repeat(blue_green_hist, len(rf))])
    """
    temp = np.array(np.meshgrid(bf, gf, rf)).T.reshape(-1,3)
    """
    print("temp:")
    print(temp.shape)

    color_hist = []    
    for i in temp:
        color_hist.append(sum(i))
    color_hist = np.array(color_hist)
    print("color hist: ")
    print(color_hist.shape)
    return color_hist

In [0]:
def split_image_into_grids(image, grid):

    grids = []

    height=image.shape[0]
    width=image.shape[1]

    M = height//grid
    N = width//grid

    for y in range(0, height, M):
        for x in range(0, width, N):
            y1 = y + M
            x1 = x + N
            grids.append(image[y:y+M, x:x+N])


    #cv2.rectangle(image, (x, y), (x1, y1), (0, 255, 0))

    #os.chdir("/content/drive/My Drive/SaveDeneme")
    #cv2.imwrite("im" + str(i) + '_' + str(x) + '_' + str(y)+".png",tiles)
    '''
    plt.figure()
    plt.imshow(grid) 
    plt.show()''' 

    return np.array(grids)

     

In [0]:
def create_dataset(path):
    print("create dataset")
    class_paths = []
    images = []
    labels = []

    with os.scandir(path) as itr: 
        for subdir in itr : 
            if subdir.is_dir():
                p = path + '/' + subdir.name
                class_paths.append(p)
    
    for p in class_paths:
        #print(p)
        for filename in os.listdir(p):
            if filename.endswith(".jpg"):
                labels.append(os.path.basename(os.path.normpath(p)))                
                img = cv2.imread(os.path.join(p, filename))

                if img is not None:
                    images.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
   
    return images, labels


In [0]:
def threaded_feature_extraction(imageslabels, grid, bin, isRGB):
    print("feature extraction")
    features = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futuretoimage = {executor.submit(feature_extract_single, imagelabel, grid, bin, isRGB): imagelabel for imagelabel in imageslabels}
        counter = 0
        for future in concurrent.futures.as_completed(futuretoimage):
            features.append(future.result())
            counter += 1
            print(counter)
    return features

def feature_extract_single(imagelabel, grid, b, isRGB):
    image = imagelabel[0]
    label = imagelabel[1]
    #print(label)
    i = cv2.resize(image, (256, 256))
    i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
    grid_hists = []
    grids = split_image_into_grids(i, grid)
    #print("grids: ")
    #print(len(grids))
    for i in grids:
        if(isRGB):
            grid_hists.append(color_histogram(i, b))
        else:
            i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)
            grid_hists.append(histogram(i, b))
    '''
    # Concatenation
    grids_flat = np.array(grid_hists).flatten()
    '''
    
    # Sum
    grids_flat = np.array(grid_hists)
    grids_flat = np.sum(grids_flat, axis=0)
    print("grids flat: ")
    print(len(grids_flat))
    return (grids_flat, label)


In [353]:
"""def feature_extraction(images, grid, bin, isRGB):
    print("feature extraction")
    features = []
    size = len(images)
    for i, j in zip(images, range(size)):
        print(j)
        i = cv2.resize(i, (256, 256))
        i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
        grids = split_image_into_grids(i, grid)

        grid_hists = []

        for i in grids:
            if(isRGB):
                grid_hists.append(color_histogram(i, bin))
            else:
                grid_hists.append(histogram(i, bin))
        
        # Concatenation
        grids_flat = np.array(grid_hists).flatten()
        
        '''
        # Sum
        grids_flat = np.array(grid_hists)
        grids_flat = np.sum(grids_flat, axis=0)
        '''
            
        features.append(grids_flat)
        
    return features"""


'def feature_extraction(images, grid, bin, isRGB):\n    print("feature extraction")\n    features = []\n    size = len(images)\n    for i, j in zip(images, range(size)):\n        print(j)\n        i = cv2.resize(i, (256, 256))\n        i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)\n        grids = split_image_into_grids(i, grid)\n\n        grid_hists = []\n\n        for i in grids:\n            if(isRGB):\n                grid_hists.append(color_histogram(i, bin))\n            else:\n                grid_hists.append(histogram(i, bin))\n        \n        # Concatenation\n        grids_flat = np.array(grid_hists).flatten()\n        \n        \'\'\'\n        # Sum\n        grids_flat = np.array(grid_hists)\n        grids_flat = np.sum(grids_flat, axis=0)\n        \'\'\'\n            \n        features.append(grids_flat)\n        \n    return features'

In [0]:
def classification(trainI, trainL, testI, testL, k):
    print("classification")
    model = KNeighborsClassifier(n_neighbors=k, metric='euclidean', algorithm='brute', n_jobs=-1)
    model.fit(trainI, trainL)
    acc = model.score(testI, testL)
    print(acc)

In [355]:
if __name__ == "__main__":

    test_path = "/content/drive/My Drive/CNG483-Project 1/TestSet"
    train_path = "/content/drive/My Drive/CNG483-Project 1/TrainingSet"
    validation_path = "/content/drive/My Drive/CNG483-Project 1/ValidationSet"

    
    # testI, testL = create_dataset(test_path)
    trainI, trainL = create_dataset(train_path)
    validationI, validationL = create_dataset(validation_path)
    
    # grid --> 1, 2, 4
    # bins --> 1, 128, 256
    # k --> 1, 5, 10
    # isRGB --> True, False

    grid = 1
    bins = 128
    isRGB = False
    k = 5

create dataset
create dataset


In [356]:
    imagelabels = []
    for i,l in zip(trainI, trainL):
        imagelabels.append((i,l))
    trainI_f = threaded_feature_extraction(imagelabels, grid, bins, isRGB)
    #trainI_f = threaded_feature_extraction(trainI, grid, bins, isRGB)
    trainf = []
    trainl = []
    for (i,l) in trainI_f:
        print(len(i))
        trainf.append(i)
        trainl.append(l)

    imagelabels = []
    for i,l in zip(validationI, validationL):
        imagelabels.append((i,l))
    
    validationI_f = threaded_feature_extraction(imagelabels, grid, bins, isRGB)
    validationf = []
    validationl = []
    for (i,l) in validationI_f:
        print(len(i))
        validationf.append(i)
        validationl.append(l)

    #validationI_f = feature_extraction(validationI, grid, bins, isRGB)
    
    trainf = np.array(trainf).reshape(-1, 1)
    trainl = np.array(trainl).reshape(-1, 1)
    validationf = np.array(validationf).reshape(-1, 1)
    validationl = np.array(validationl).reshape(-1, 1)


feature extraction
grids flat: 
128
grids flat: 
128
grids flat: 
116
grids flat: grids flat: 
128
grids flat: 

127
128
grids flat: 
116
grids flat: 
127
grids flat: grids flat: 
127

128
grids flat: 
128
grids flat: 
128
grids flat: 
128
grids flat: 
126
1
2
grids flat: 3grids flat: 
127
grids flat: 
127
grids flat: 
grids flat: 
124
grids flat: 
grids flat: 128

grids flat: 
128

128
grids flat: 
128
grids flat: 4125


120
128
5
6grids flat: 
121


grids flat: 
128
grids flat: grids flat: grids flat: 
128
grids flat: 
128
grids flat: 
121
grids flat: 
120grids flat: 
124
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21grids flat: 
grids flat: 
128

128

grids flat: 
128

22
23
24
25
26
27
28
29
30
31
32
33
34
128
grids flat: 
128
grids flat: 
128
35
36
37
128
grids flat: grids flat: 
120
grids flat: 
128

128
grids flat: grids flat: 
grids flat: 
128
grids flat: grids flat: 
128grids flat: grids flat: 
128

127
grids flat: 
127

128

128
100
grids flat: 

grids flat: 
128
grids flat: 
124


In [357]:
    validationf


array([[array([    1,     1,     8,    19,    35,    64,   122,   184,   242,
         313,   389,   506,   509,   596,   547,   562,   529,   514,
         481,   461,   461,   399,   461,   450,   437,   415,   403,
         428,   416,   442,   432,   446,   467,   435,   480,   488,
         536,   487,   548,   563,   492,   502,   425,   395,   417,
         336,   378,   344,   367,   372,   420,   465,   484,   536,
         636,   761,   763,   790,   641,   510,   430,   408,   328,
         295,   307,   320,   318,   399,   359,   327,   410,   446,
         487,   530,   584,   619,   595,   526,   472,   458,   437,
         481,   462,   454,   519,   460,   496,   455,   367,   401,
         362,   359,   305,   305,   295,   307,   296,   296,   297,
         325,   325,   299,   326,   351,   356,   332,   264,   297,
         339,   315,   328,   210,   246,   236,   188,   232,   242,
         203,   191,    90,    41,    14,     7,    11,    66,   382,
        1806

In [358]:
validationf.tolist()

[[array([    1,     1,     8,    19,    35,    64,   122,   184,   242,
           313,   389,   506,   509,   596,   547,   562,   529,   514,
           481,   461,   461,   399,   461,   450,   437,   415,   403,
           428,   416,   442,   432,   446,   467,   435,   480,   488,
           536,   487,   548,   563,   492,   502,   425,   395,   417,
           336,   378,   344,   367,   372,   420,   465,   484,   536,
           636,   761,   763,   790,   641,   510,   430,   408,   328,
           295,   307,   320,   318,   399,   359,   327,   410,   446,
           487,   530,   584,   619,   595,   526,   472,   458,   437,
           481,   462,   454,   519,   460,   496,   455,   367,   401,
           362,   359,   305,   305,   295,   307,   296,   296,   297,
           325,   325,   299,   326,   351,   356,   332,   264,   297,
           339,   315,   328,   210,   246,   236,   188,   232,   242,
           203,   191,    90,    41,    14,     7,    11,    66,

In [359]:
    print(trainf.dtype)
    print(validationf.dtype)
    print(trainl.dtype)

object
object
<U10


In [360]:
    classification(trainf, trainl, validationf, validationl, k)
    output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

classification


ValueError: ignored